In [1]:
from model_training import GPTConfig, train, GPT

using device: cuda


In [2]:
# CODE UPDATE HERE
max_lr = 6e-4 
min_lr = max_lr * 0.1
warmup_steps = 50
max_steps = 5000

In [3]:
gpt_config = GPTConfig()
gpt_config.B = 16

In [4]:
print("Starting training")
train(warmup_steps=warmup_steps,
      max_steps=max_steps,
      min_lr=min_lr,
      max_lr=max_lr, 
      gpt_config=gpt_config)

Starting training
loaded 338025 tokens
1 epoch = 20 batches
num decayed parameter tensors: 50, with 124,354,560 parameters
num non-decayed parameter tensors: 98, with 121,344 parameters
using fused AdamW: True


c:\Users\1430647\My\repo\github\assignment-21\model_training.py:300: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  norm = torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)


step0 | loss: 10.97528076171875 | dt: 3255.92ms | tok/sec:  5032.06 | norm: 7.12
step1 | loss: 10.61593246459961 | dt: 1933.75ms | tok/sec:  8472.64 | norm: 7.63
step2 | loss: 10.125922203063965 | dt: 1783.32ms | tok/sec:  9187.36 | norm: 5.60
step3 | loss: 9.684972763061523 | dt: 1746.37ms | tok/sec:  9381.75 | norm: 4.45
step4 | loss: 9.318449020385742 | dt: 1727.77ms | tok/sec:  9482.72 | norm: 3.42
step5 | loss: 9.230635643005371 | dt: 1731.73ms | tok/sec:  9461.08 | norm: 2.68
step6 | loss: 9.183297157287598 | dt: 1780.93ms | tok/sec:  9199.67 | norm: 3.13
step7 | loss: 9.035476684570312 | dt: 1815.76ms | tok/sec:  9023.21 | norm: 2.76
step8 | loss: 8.819440841674805 | dt: 1732.73ms | tok/sec:  9455.59 | norm: 2.57
step9 | loss: 8.713406562805176 | dt: 1775.68ms | tok/sec:  9226.90 | norm: 2.58
step10 | loss: 8.550621032714844 | dt: 1894.83ms | tok/sec:  8646.69 | norm: 3.24
step11 | loss: 8.46468734741211 | dt: 1707.33ms | tok/sec:  9596.29 | norm: 2.16
step12 | loss: 8.307191848

In [5]:
import tiktoken
import torch
enc = tiktoken.get_encoding('gpt2') 


In [6]:
import os
model = GPT(gpt_config)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
out_dir = 'out'
model.load_state_dict(torch.load(os.path.join(out_dir, 'ckpt.pt')))
model = model.to(device)

In [7]:
num_return_sequences = 5
max_length = 30

In [8]:

text = "what services he has done for his country ?"
tokens = enc.encode(text)
tokens = torch.tensor(tokens)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
x = tokens.to('cuda')

In [9]:
from torch.nn import functional as F

def generate_output(model, input_tokens, num_return_sequences, max_length):
    x = input_tokens
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)
    while x.size(1) < max_length:
        # forward the model to get the logits
        with torch.no_grad():
            logits = model(x)[0] # (B, T, vocab_size)
            # take the logits at the last position
            logits = logits[:, -1, :] # (B, vocab_size)
            # get the probabilities
            probs = F.softmax(logits, dim=-1)
            # do top-k sampling of 50 (huggingface pipeline default)
            # topk_probs here becomes (5, 50), topk_indices is (5, 50)
            topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
            # select a token from the top-k probabilities
            # note: multinomial does not demand the input to sum to 1
            ix = torch.multinomial(topk_probs, 1) # (B, 1)
            # gather the corresponding indices
            xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
            # append to the sequence
            x = torch.cat((x, xcol), dim=1)

    output = []
    # print the generated text
    for i in range(num_return_sequences):
        tokens = x[i, :max_length].tolist()
        output.append(tokens)
        # decoded = enc.decode(tokens)

        # print(">", decoded)
    return output

In [10]:
results = generate_output(model=model, input_tokens=x,num_return_sequences=5,max_length=max_length)

In [11]:
for result in results:
    decoded = enc.decode(result)
    print(f">{decoded}")

>what services he has done for his country ?


To tell,ue with
, and excuses too is honour to be
W intellect,
>what services he has done for his country ? is with?
L'll find to our his bitter head undone!
That,
Why, I
>what services he has done for his country ? is let?
 o' my to both.


On yet oft die to me,

>what services he has done for his country ?

More eF lie tonight
dayTwaning me
is thought not,
.
 these
>what services he has done for his country ?

 strong?

 way in the outward Elizabeth, too may be name be
More light down
